In [14]:
import numpy as np
import operator

In [4]:
def createDataSet():
    group = np.array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group, labels

createDataSet()

(array([[1. , 1.1],
        [1. , 1. ],
        [0. , 0. ],
        [0. , 0.1]]), ['A', 'A', 'B', 'B'])

In [10]:
group = createDataSet()[0].shape[0]
print(group)

4


此处用来解释下面一些numpy函数的作用。
我使用了这个网址用来简化latex符号的编写，很方便。
[在线制作latex数学符号](http://latex.codecogs.com/eqneditor/editor.php)
### tile函数
tile函数是用来重复输出，比如下面的欧式距离公式
$$\sqrt{\sum_{i=1}^{n}\left ( x_{i}-y_{i} \right )^{2} }$$

我们的输入向量现在只有一个，所以要扩充输入向量使其纬度与样本矩阵相同，然后矩阵相减。

一般来说输入向量的列数与样本矩阵相同。

如输入矩阵: $$\begin{bmatrix}
 1& 2\\ 
\end{bmatrix}$$

样本矩阵: $$\begin{bmatrix}
1 & 2\\ 
3 &4 \\ 
5 & 6\\ 
7 & 8
\end{bmatrix}$$

我们需要扩充输入矩阵使其和样本矩阵的纬度相同，扩展成如下形式：

$$\begin{bmatrix}
1 & 2\\ 
1 & 2 \\ 
1 & 2\\ 
1 & 2
\end{bmatrix}$$

然后才能计算两个矩阵的欧式距离，具体到numpy的语法如下。

In [23]:
def classify0(inx, dataSet, labels, k):  # 取前k个距离最小的点，小于样本的分类
    dataSetSize = dataSet.shape[0]  # 得到样本数量
    diffMat = np.tile(inx, (dataSetSize, 1)) - dataSet  # 扩充输入向量并减去样本矩阵
    sqDiffMat = diffMat ** 2  # 数据差取平方
    # 当axis为0时,是压缩行,即将每一列的元素相加,将矩阵压缩为一行 
    # 当axis为1时,是压缩列,即将每一行的元素相加,将矩阵压缩为一列
    sqDistances = sqDiffMat.sum(axis=1)  
    distances = sqDistances ** 0.5  # 结果矩阵开根号即对每个结果元素开跟
    sorteDisIndicies = distances.argsort()  # 结果升序排序，返回排序后的索引
    classCount = dict()
    for i in range(k):
        voteIlabel = labels[sorteDisIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1  # 计算前k个点所在类别的频率
    
#     operator模块提供的itemgetter函数用于获取对象的哪些维的数据，参数为一些序号（即需要获取的数据在对象中的序号），
#     下面看例子。
#     a = [1,2,3] 
#     >>> b=operator.itemgetter(1)      //定义函数b，获取对象的第1个域的值
#     >>> b(a) 
#     2 
#     >>> b=operator.itemgetter(1,0)   //定义函数b，获取对象的第1个域和第0个的值
#     >>> b(a) 
#     (2, 1) 

#     要注意，operator.itemgetter函数获取的不是值，而是定义了一个函数，通过该函数作用到对象上才能获取值。
    
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)  
    # Python内置的排序函数sorted可以对list或者iterator进行排序
    return sortedClassCount[:k]
        
dataSet, labels = createDataSet()

classify0(np.array([1,2]), dataSet, labels, 1)

[('A', 1)]

# 实例：在约会网站上使用k-近邻算法

In [24]:
def file2matrix(filename):
    with open(filename) as fr:
        arrayOLines = fr.readlines()
        numberOfLines = len(arrayOLines)
        returnMat = np.zeros((numberOfLines, 3))
        classLabelVector = list()
        index = 0
        for line in arrayOLines:
            line = line.strip()
            listFormLine = line.split("\t")
            returnMat[index,:] = listFormLine[0:3]
            classLabelVector.append(int(listFormLine[-1]))
            index += 1
        return returnMat, classLabelVector